In [15]:
cafe = bytes('cafe', encoding='utf-8')
print(cafe)

for i,b in enumerate(cafe) : 
    print(f"cafe[{i}] : {b}")
    
cafe_arr = bytearray('cafe', encoding='utf-8')
print("\n",cafe_arr, sep = '')

for i,b in enumerate(cafe_arr) : 
    print(f"cafe[{i}] : {b}")
    
    
# bytearray[:1]은 bytearray를 반환한다.
print("\n", cafe_arr[:1], sep = '')

# s[0] == s[:1] 이 참이 되는 경우는 str이 유일하다.


b'cafe'
cafe[0] : 99
cafe[1] : 97
cafe[2] : 102
cafe[3] : 101

bytearray(b'cafe')
cafe[0] : 99
cafe[1] : 97
cafe[2] : 102
cafe[3] : 101

bytearray(b'c')


In [17]:
# bytes는 형변환을 지원한다.

import array
nums = array.array('h', [-2,-1,0,1,2])
octets = bytes(nums)
octets

b'\xfe\xff\xff\xff\x00\x00\x01\x00\x02\x00'

In [27]:
# 어떤 codec을 사용하냐에 따라 인코딩이 달라진다.

for codec in ['utf_8', 'utf_16', 'cp949'] :
    print(codec, "사과".encode(codec), sep = '\t')

utf_8	b'\xec\x82\xac\xea\xb3\xbc'
utf_16	b'\xff\xfe\xac\xc0\xfc\xac'
cp949	b'\xbb\xe7\xb0\xfa'


In [37]:
#파일을 작성할 때 작성한 codec과 읽을 때 사용한 codec이 다르면
#에러를 발생하거나 문자가 깨질 수 있다.
#따라서 codec을 미리 통일할 필요가 있다. (특히 여러 사람과 작업하게 된다면)

with open("text.txt", 'w', encoding = 'cp949') as f :
    f.write("안녕하세요.")

try :
    f = open("text.txt", 'r', encoding = 'utf-8').read()
except Exception as e : 
    print(e)
    f = open("text.txt", 'r', encoding = 'cp949').read()
    
print(f)

'utf-8' codec can't decode byte 0xbe in position 0: invalid start byte
안녕하세요.


In [68]:
# sys와 locale 에서 default codec을 확인할 수 있다.

# locale은 현재 코드 작성 환경의 codec을 보여준다. 
# sys 은 표준입출력의 codec을 보여준다.
# sys.~.isatty() 은 입출력이 콘솔(터미널) 에서 작동하는지 여부를 알려준다.

# 출력을 보면 현재 필자의 코드 작성 환경이 주피터 노트북으로 리다이렉션 됬으므로 isatty() 가 False로 나온다.
#   리다이렉션 : 표준입출력을 사용자 지정 위치로 우회하는 것을 말한다.
# locale 이 cp949 이므로 my_file도 cp949로 encoding 되었음을 알 수 있다.
# 하지만 sys 상에서는 utf-8을 디폴트로 사용하고 있다.
# 따라서 my_file을 주피터 노트북 환경이 아닌 터미널에서 실행하여 파일을 읽어들일 경우 오류가 발생할 것을 예상할 수 있다.
# 이 경우에는 encoding = 'cp949' 을 설정해야 오류를 피할 수 있다.

# 결론 : 기본 인코딩에 의존하지 말고 인코딩을 명시하자.

import sys, locale

expressions = """ 
        locale.getprefererdencoding()
        type(my_file)
        my_file.encoding
        sys.stdout.isatty()
        sys.stdout.encoding
        sys.stdin.isatty()
        sys.stdin.encoding
        sys.stderr.isatty()
        sys.getdefaultencoding()
        sys.getfilesystemencoding()
"""

my_file = open('dummy', 'w')

for expression in expressions.split() :
    value = eval(expression)
    print(f"{expression:>30} -> {str(value)}")
    
    


 locale.getpreferredencoding() -> cp949
                 type(my_file) -> <class '_io.TextIOWrapper'>
              my_file.encoding -> cp949
           sys.stdout.isatty() -> False
           sys.stdout.encoding -> UTF-8
            sys.stdin.isatty() -> False
            sys.stdin.encoding -> utf-8
           sys.stderr.isatty() -> False
      sys.getdefaultencoding() -> utf-8
   sys.getfilesystemencoding() -> utf-8
